In [ ]:
# !pip install git+https://github.com/huggingface/transformers
!pip install transformers
!pip install accelerate
!pip install bitsandbytes



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, pipeline, TextStreamer, BitsAndBytesConfig

if torch.is_device_available('cuda'):
  device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=False, # set to True to save more VRAM at the cost of some speed/accuracy
    )
    )
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [ ]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
    # {'role': 'user', ''}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
# model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=128, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


In [ ]:
# other method:

import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, pipeline, TextStreamer, BitsAndBytesConfig

tokenizer = LlamaTokenizer.from_pretrained("kittn/mistral-7B-v0.1-hf")
model = LlamaForCausalLM.from_pretrained(
    "kittn/mistral-7B-v0.1-hf",
    device_map={"": 0},
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=False, # set to True to save more VRAM at the cost of some speed/accuracy
    ),
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

pipe("Hi, my name", streamer=TextStreamer(tokenizer), max_new_tokens=128)
